In [2]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
import pandas as pd
import numpy as np
import os
import csv

In [3]:
mortality_with_sundown_census_poverty = pd.read_csv('../data/mortality_with_sundown_census_poverty.csv')

/usr/local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (9,11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
mortality_with_sundown_census_poverty = mortality_with_sundown_census_poverty.rename(
    columns={
        'All Ages in Poverty Percent': 'perc_poverty',
        'Median Household Income in Dollars': 'median_income_str'
    })

In [5]:
df_sundown = pd.read_csv('../data/sundown_with_counties.csv')
analysis_df = pd.merge(mortality_with_sundown_census_poverty, df_sundown, how='left', left_on='County', right_on="county")

In [6]:
# pull out state
analysis_df['state'] = analysis_df.County.apply(lambda x: x.split(', ')[1])

In [7]:
# merge in state shootings data
state_shootings = pd.read_csv('../data/police-killings-state.csv')
analysis_df = pd.merge(analysis_df, state_shootings, left_on='state', right_on='State Abbreviation', how='left')

In [8]:
# layer in jim crow states
jim_crow_states = [
    'TX', 'OK', 'MO', 'AR', 'LA', 'KY', 'WV', 'MD', 'VA', 'TN', 'NC', 'MS', 'AL', 'GA', 'SC', 'FL'
]
analysis_df['is_jim_crow_state'] = analysis_df.state.apply(lambda x: x in jim_crow_states)
analysis_df['num_sundown_towns'] = analysis_df['#']


In [9]:
# layer in state racism index
racism_index = pd.read_csv('../data/racism_index.csv')
analysis_df = pd.merge(analysis_df, racism_index, left_on='state', right_on='state_abbr', how='left')

In [10]:
# Merge in city shootings df
city_shootings = pd.read_csv('../data/police-killings-city.csv')
analysis_df = pd.merge(analysis_df, city_shootings, left_on='County', right_on='county', how='left')

In [21]:
analysis_df_with_city_shootings = analysis_df.dropna(subset=['Killings by Police per 10k Arrests'])

analysis_df_with_city_shootings['killings_per_10k'] = analysis_df_with_city_shootings['Killings by Police per 10k Arrests']
analysis_df_with_city_shootings['diss_index'] = analysis_df_with_city_shootings['Black-White Dissimilarity Index (2010)']
analysis_df_with_city_shootings['num_sundown_towns'] = analysis_df_with_city_shootings['#']
analysis_df_with_city_shootings['perc_white'] = 1 - analysis_df_with_city_shootings['perc_minority']

# coerce White, Black to int
analysis_df_with_city_shootings['White_pop'] = analysis_df_with_city_shootings['White'].apply(lambda x: int(x.replace(',', '')))
analysis_df_with_city_shootings['Black_pop'] = analysis_df_with_city_shootings['Black'].apply(lambda x: int(x.replace(',', '')))
analysis_df_with_city_shootings['black_to_white_shootings_ratio'] = (analysis_df_with_city_shootings['Black People Killed by Police (1/1/2013-12/31/2019)'] / analysis_df_with_city_shootings['Black_pop']) / (analysis_df_with_city_shootings['White People Killed by Police (1/1/2013-12/31/2019)'] / analysis_df_with_city_shootings['white_pop'])

analysis_df_with_city_shootings = analysis_df_with_city_shootings.dropna(subset=['black_to_white_shootings_ratio'])

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value inst

In [22]:
analysis_df['median_income'] = analysis_df.median_income_str.apply(lambda x: float(x.replace('$', '').replace(',', ''))/10000)
analysis_df_with_city_shootings['median_income'] = analysis_df_with_city_shootings.median_income_str.apply(lambda x: float(x.replace('$', '').replace(',', ''))/10000)

# Analysis using full dataframe (full analysis_df)

Excluding jim crow status

In [13]:
# analysis using full dataframe

y = analysis_df['Death_Population_Ratio']
logit_covariates = [
    'perc_minority',
    'num_sundown_towns',
    'perc_poverty',
    'median_income',
]
X = analysis_df[logit_covariates]
X = X.convert_objects(convert_numeric=True)
y = y.convert_objects(convert_numeric=True)

ols_model = sm.OLS(y.astype(float), X.astype(float))
ols_result = ols_model.fit()
ols_result.summary2()

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:11: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use DataFrame.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:12: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use Series.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  if sys.path[0] == '':


<class 'statsmodels.iolib.summary2.Summary'>
"""
                             Results: Ordinary least squares
=========================================================================================
Model:                  OLS                    Adj. R-squared (uncentered): 0.922        
Dependent Variable:     Death_Population_Ratio AIC:                         -3304716.5148
Date:                   2020-12-08 22:36       BIC:                         -3304672.9762
No. Observations:       394212                 Log-Likelihood:              1.6524e+06   
Df Model:               4                      F-statistic:                 1.168e+06    
Df Residuals:           394208                 Prob (F-statistic):          0.00         
R-squared (uncentered): 0.922                  Scale:                       1.3390e-05   
---------------------------------------------------------------------------------------------
                          Coef.      Std.Err.        t         P>|t|       [0.025      0.975]
---------------------------------------------------------------------------------------------
perc_minority            -0.0011       0.0000     -25.5127     0.0000     -0.0012     -0.0010
num_sundown_towns         0.0000       0.0000       8.7437     0.0000      0.0000      0.0000
perc_poverty              0.0005       0.0000     588.9716     0.0000      0.0005      0.0005
median_income             0.0009       0.0000     493.4205     0.0000      0.0009      0.0009
-----------------------------------------------------------------------------------------
Omnibus:                    15986.203             Durbin-Watson:                0.005    
Prob(Omnibus):              0.000                 Jarque-Bera (JB):             49837.966
Skew:                       0.092                 Prob(JB):                     0.000    
Kurtosis:                   4.732                 Condition No.:                133      
=========================================================================================

"""

Include Jim Crow status

In [19]:
# analysis using full dataframe

y = analysis_df['Death_Population_Ratio']
logit_covariates = [
    'perc_minority',
    'num_sundown_towns',
    'perc_poverty',
    'is_jim_crow_state',
    'median_income',
]
X = analysis_df[logit_covariates]
X = X.convert_objects(convert_numeric=True)
y = y.convert_objects(convert_numeric=True)

ols_model = sm.OLS(y.astype(float), X.astype(float))
ols_result = ols_model.fit()

ols_result.summary2()

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:12: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use DataFrame.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  if sys.path[0] == '':
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:13: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use Series.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  del sys.path[0]


<class 'statsmodels.iolib.summary2.Summary'>
"""
                             Results: Ordinary least squares
=========================================================================================
Model:                  OLS                    Adj. R-squared (uncentered): 0.923        
Dependent Variable:     Death_Population_Ratio AIC:                         -3309818.3120
Date:                   2020-12-08 22:40       BIC:                         -3309763.8888
No. Observations:       394212                 Log-Likelihood:              1.6549e+06   
Df Model:               5                      F-statistic:                 9.477e+05    
Df Residuals:           394207                 Prob (F-statistic):          0.00         
R-squared (uncentered): 0.923                  Scale:                       1.3218e-05   
---------------------------------------------------------------------------------------------
                          Coef.      Std.Err.        t         P>|t|       [0.025      0.975]
---------------------------------------------------------------------------------------------
perc_minority            -0.0017       0.0000     -38.6064     0.0000     -0.0018     -0.0016
num_sundown_towns         0.0000       0.0000      40.2766     0.0000      0.0000      0.0000
perc_poverty              0.0005       0.0000     525.6627     0.0000      0.0005      0.0005
is_jim_crow_state         0.0010       0.0000      71.6723     0.0000      0.0010      0.0011
median_income             0.0008       0.0000     480.1724     0.0000      0.0008      0.0008
-----------------------------------------------------------------------------------------
Omnibus:                    13898.933             Durbin-Watson:                0.005    
Prob(Omnibus):              0.000                 Jarque-Bera (JB):             39274.166
Skew:                       0.091                 Prob(JB):                     0.000    
Kurtosis:                   4.536                 Condition No.:                136      
=========================================================================================

"""

In [20]:
ols_result.pvalues

perc_minority        0.0
num_sundown_towns    0.0
perc_poverty         0.0
is_jim_crow_state    0.0
median_income        0.0
dtype: float64

# Analysis using just cities we have shootings data for

1. regress (minority%, # of sundown town, perc of poverty, jim crow state, median income) -> killings by police

In [23]:
y = analysis_df_with_city_shootings['black_to_white_shootings_ratio']
logit_covariates = [
    'perc_minority',
    '#',
    'perc_poverty',
    'is_jim_crow_state',
    'median_income',
]
X = analysis_df_with_city_shootings[logit_covariates]
X = X.convert_objects(convert_numeric=True)
y = y.convert_objects(convert_numeric=True)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:10: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use DataFrame.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:11: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use Series.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  # This is added back by InteractiveShellApp.init_path()


In [24]:
ols_model = sm.OLS(y.astype(float), X.astype(float))
ols_result = ols_model.fit()

In [25]:
ols_result.summary2()

<class 'statsmodels.iolib.summary2.Summary'>
"""
                                Results: Ordinary least squares
===============================================================================================
Model:                  OLS                            Adj. R-squared (uncentered): 0.099      
Dependent Variable:     black_to_white_shootings_ratio AIC:                         796177.5602
Date:                   2020-12-09 21:36               BIC:                         796223.8664
No. Observations:       77748                          Log-Likelihood:              -3.9808e+05
Df Model:               5                              F-statistic:                 1713.      
Df Residuals:           77743                          Prob (F-statistic):          0.00       
R-squared (uncentered): 0.099                          Scale:                       1640.2     
----------------------------------------------------------------------------------------------------
                            Coef.       Std.Err.         t          P>|t|        [0.025       0.975]
----------------------------------------------------------------------------------------------------
perc_minority              -5.3638        1.6369       -3.2768      0.0011      -8.5721      -2.1555
#                           0.4003        0.0171       23.4293      0.0000       0.3668       0.4337
perc_poverty               -0.8859        0.0326      -27.1494      0.0000      -0.9498      -0.8219
is_jim_crow_state          -3.7203        0.4273       -8.7056      0.0000      -4.5579      -2.8827
median_income               3.2668        0.0639       51.1515      0.0000       3.1417       3.3920
-----------------------------------------------------------------------------------------------
Omnibus:                     134761.866             Durbin-Watson:                0.770        
Prob(Omnibus):               0.000                  Jarque-Bera (JB):             121820664.533
Skew:                        12.395                 Prob(JB):                     0.000        
Kurtosis:                    195.329                Condition No.:                221          
===============================================================================================

"""

2.  Regression with JIM Crow + Sundowntowns + % minority + median_income + perc_poverty → Black White Dissimilarity Index (2010)

In [19]:
y = analysis_df_with_city_shootings['Black-White Dissimilarity Index (2010)']
logit_covariates = [
    '#',
    'is_jim_crow_state',
]
X = analysis_df_with_city_shootings[logit_covariates]
X = X.convert_objects(convert_numeric=True)
y = y.convert_objects(convert_numeric=True)

ols_model = sm.OLS(y.astype(float), X.astype(float))
ols_result = ols_model.fit()
ols_result.summary2()

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:7: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use DataFrame.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  import sys
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:8: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use Series.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  


<class 'statsmodels.iolib.summary2.Summary'>
"""
                                    Results: Ordinary least squares
=======================================================================================================
Model:                  OLS                                    Adj. R-squared (uncentered): 0.645      
Dependent Variable:     Black-White Dissimilarity Index (2010) AIC:                         774706.9453
Date:                   2020-12-03 08:54                       BIC:                         774725.4677
No. Observations:       77748                                  Log-Likelihood:              -3.8735e+05
Df Model:               2                                      F-statistic:                 7.062e+04  
Df Residuals:           77746                                  Prob (F-statistic):          0.00       
R-squared (uncentered): 0.645                                  Scale:                       1244.4     
--------------------------------------------------------------------------------------------------------------
                            Coef.         Std.Err.           t            P>|t|          [0.025         0.975]
--------------------------------------------------------------------------------------------------------------
#                           3.1268          0.0093        337.1340        0.0000         3.1086         3.1450
is_jim_crow_state          45.0082          0.3283        137.0929        0.0000        44.3648        45.6517
-------------------------------------------------------------------------------------------------------
Omnibus:                          7901.686                  Durbin-Watson:                     0.093   
Prob(Omnibus):                    0.000                     Jarque-Bera (JB):                  6881.886
Skew:                             0.651                     Prob(JB):                          0.000   
Kurtosis:                         2.346                     Condition No.:                     36      
=======================================================================================================

"""

In [20]:
y = analysis_df_with_city_shootings['Black-White Dissimilarity Index (2010)']
logit_covariates = [
    '#',
    'is_jim_crow_state',
    'median_income',
]
X = analysis_df_with_city_shootings[logit_covariates]
X = X.convert_objects(convert_numeric=True)
y = y.convert_objects(convert_numeric=True)

ols_model = sm.OLS(y.astype(float), X.astype(float))
ols_result = ols_model.fit()
ols_result.summary2()

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:8: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use DataFrame.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:9: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use Series.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  if __name__ == '__main__':


<class 'statsmodels.iolib.summary2.Summary'>
"""
                                    Results: Ordinary least squares
=======================================================================================================
Model:                  OLS                                    Adj. R-squared (uncentered): 0.832      
Dependent Variable:     Black-White Dissimilarity Index (2010) AIC:                         716451.4719
Date:                   2020-12-03 08:54                       BIC:                         716479.2556
No. Observations:       77748                                  Log-Likelihood:              -3.5822e+05
Df Model:               3                                      F-statistic:                 1.285e+05  
Df Residuals:           77745                                  Prob (F-statistic):          0.00       
R-squared (uncentered): 0.832                                  Scale:                       588.24     
--------------------------------------------------------------------------------------------------------------
                            Coef.         Std.Err.           t            P>|t|          [0.025         0.975]
--------------------------------------------------------------------------------------------------------------
#                           0.9116          0.0099         92.4451        0.0000         0.8923         0.9309
is_jim_crow_state          13.1118          0.2504         52.3717        0.0000        12.6211        13.6025
median_income               6.3932          0.0217        294.4960        0.0000         6.3507         6.4358
-------------------------------------------------------------------------------------------------------
Omnibus:                          3399.171                  Durbin-Watson:                     0.197   
Prob(Omnibus):                    0.000                     Jarque-Bera (JB):                  3092.288
Skew:                             -0.431                    Prob(JB):                          0.000   
Kurtosis:                         2.540                     Condition No.:                     42      
=======================================================================================================

"""

3. regress (minority%, # of sundown town, sundown county status, perc of poverty, jim crow state, median income) -> ratio of blacks / whites killings by police

In [21]:
y = analysis_df_with_city_shootings['black_to_white_shootings_ratio']
logit_covariates = [
    'perc_minority',
    '#',
    'is_jim_crow_state',
]
X = analysis_df_with_city_shootings[logit_covariates]
X = X.convert_objects(convert_numeric=True)
y = y.convert_objects(convert_numeric=True)

ols_model = sm.OLS(y.astype(float), X.astype(float))
ols_result = ols_model.fit()
ols_result.summary2()

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:8: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use DataFrame.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:9: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use Series.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  if __name__ == '__main__':


<class 'statsmodels.iolib.summary2.Summary'>
"""
                               Results: Ordinary least squares
==============================================================================================
Model:                  OLS                            Adj. R-squared (uncentered): 0.401     
Dependent Variable:     black_to_white_shootings_ratio AIC:                         23613.3938
Date:                   2020-12-03 08:54               BIC:                         23641.1775
No. Observations:       77748                          Log-Likelihood:              -11804.   
Df Model:               3                              F-statistic:                 1.734e+04 
Df Residuals:           77745                          Prob (F-statistic):          0.00      
R-squared (uncentered): 0.401                          Scale:                       0.079325  
---------------------------------------------------------------------------------------------------
                           Coef.       Std.Err.         t          P>|t|        [0.025       0.975]
---------------------------------------------------------------------------------------------------
perc_minority              0.7332        0.0057      127.6636      0.0000       0.7219       0.7444
#                         -0.0011        0.0001       -9.7739      0.0000      -0.0013      -0.0009
is_jim_crow_state          0.1700        0.0029       58.6934      0.0000       0.1643       0.1757
----------------------------------------------------------------------------------------------
Omnibus:                      53585.664              Durbin-Watson:                 0.055     
Prob(Omnibus):                0.000                  Jarque-Bera (JB):              797937.542
Skew:                         3.210                  Prob(JB):                      0.000     
Kurtosis:                     17.321                 Condition No.:                 80        
==============================================================================================

"""

4. Add interaction term: white_perc * diss_index

In [22]:
y = analysis_df_with_city_shootings['black_to_white_shootings_ratio']
logit_covariates = [
    'perc_minority',
    'median_income',
    '#',
    'is_jim_crow_state',
#     'Black-White Dissimilarity Index (2010)',
]
X = analysis_df_with_city_shootings[logit_covariates]
X = X.convert_objects(convert_numeric=True)
y = y.convert_objects(convert_numeric=True)

ols_model = sm.OLS(y.astype(float), X.astype(float))
ols_result = ols_model.fit()
ols_result.summary2()

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:10: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use DataFrame.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:11: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use Series.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  # This is added back by InteractiveShellApp.init_path()


<class 'statsmodels.iolib.summary2.Summary'>
"""
                               Results: Ordinary least squares
==============================================================================================
Model:                  OLS                            Adj. R-squared (uncentered): 0.470     
Dependent Variable:     black_to_white_shootings_ratio AIC:                         14140.5779
Date:                   2020-12-03 08:54               BIC:                         14177.6229
No. Observations:       77748                          Log-Likelihood:              -7066.3   
Df Model:               4                              F-statistic:                 1.721e+04 
Df Residuals:           77744                          Prob (F-statistic):          0.00      
R-squared (uncentered): 0.470                          Scale:                       0.070225  
---------------------------------------------------------------------------------------------------
                           Coef.       Std.Err.         t          P>|t|        [0.025       0.975]
---------------------------------------------------------------------------------------------------
perc_minority              0.0329        0.0088        3.7327      0.0002       0.0156       0.0502
median_income              0.0389        0.0004      100.3783      0.0000       0.0381       0.0396
#                         -0.0043        0.0001      -38.6651      0.0000      -0.0045      -0.0040
is_jim_crow_state          0.1262        0.0028       45.7142      0.0000       0.1207       0.1316
----------------------------------------------------------------------------------------------
Omnibus:                      53739.805              Durbin-Watson:                 0.060     
Prob(Omnibus):                0.000                  Jarque-Bera (JB):              806042.794
Skew:                         3.220                  Prob(JB):                      0.000     
Kurtosis:                     17.399                 Condition No.:                 136       
==============================================================================================

"""

5. Regression with JIM + Sundown towns + **percentage white+ Black White Dissimilarity Index (2010) → Racism index (2013-2017) in state racism index 2013-2017 file

In [23]:
y = analysis_df_with_city_shootings['racism_index']
logit_covariates = [
    'perc_minority',
    'median_income',
    '#',
    'is_jim_crow_state',
]
X = analysis_df_with_city_shootings[logit_covariates]
X = X.convert_objects(convert_numeric=True)
y = y.convert_objects(convert_numeric=True)

ols_model = sm.OLS(y.astype(float), X.astype(float))
ols_result = ols_model.fit()
ols_result.summary2()

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:9: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use DataFrame.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  if __name__ == '__main__':
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:10: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use Series.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  # Remove the CWD from sys.path while we load stuff.


<class 'statsmodels.iolib.summary2.Summary'>
"""
                         Results: Ordinary least squares
=================================================================================
Model:                  OLS              Adj. R-squared (uncentered): 0.956      
Dependent Variable:     racism_index     AIC:                         604150.6515
Date:                   2020-12-03 08:54 BIC:                         604187.6964
No. Observations:       77748            Log-Likelihood:              -3.0207e+05
Df Model:               4                F-statistic:                 4.268e+05  
Df Residuals:           77744            Prob (F-statistic):          0.00       
R-squared (uncentered): 0.956            Scale:                       138.75     
------------------------------------------------------------------------------------
                      Coef.     Std.Err.       t        P>|t|      [0.025     0.975]
------------------------------------------------------------------------------------
perc_minority        56.5522      0.3922    144.1812    0.0000    55.7834    57.3209
median_income         5.5358      0.0172    321.5183    0.0000     5.5020     5.5695
#                     0.4037      0.0049     82.3186    0.0000     0.3941     0.4133
is_jim_crow_state    -7.7471      0.1227    -63.1554    0.0000    -7.9875    -7.5067
---------------------------------------------------------------------------------
Omnibus:                   965.406           Durbin-Watson:              0.008   
Prob(Omnibus):             0.000             Jarque-Bera (JB):           1515.343
Skew:                      0.120             Prob(JB):                   0.000   
Kurtosis:                  3.640             Condition No.:              136     
=================================================================================

"""

In [24]:
## Boxplots

In [59]:
unique_fips = analysis_df_with_city_shootings.fips_str.unique()
a = analysis_df_with_city_shootings
sundown_towns_dist = a.loc[a.fips_str.apply(lambda x: x in unique_fips)]
sundown_towns_dist

,Unnamed: 0_x,County,County Code,Deaths,Population,Death_Population_Ratio,Race,fips,fips_str,county_x,...,2018 Total Arrests,Estimated Average Arrests per Year,Killings by Police per 10k Arrests,county,killings_per_10k,diss_index,White_pop,Black_pop,black_to_white_shootings_ratio,median_income
12314,12312,"Maricopa County, AZ",4013.0,813.0,73697.0,0.011032,Black,4013,4013,Maricopa County AZ,...,52449.0,51638.25,21.5,"Maricopa County, AZ",21.5,49.89,672573,86788,0.344103,6.5234
12315,12312,"Maricopa County, AZ",4013.0,813.0,73697.0,0.011032,Black,4013,4013,Maricopa County AZ,...,9296.0,9312.8,8.6,"Maricopa County, AZ",8.6,23.46,182011,3484,0.076567,6.5234
12318,12312,"Maricopa County, AZ",4013.0,813.0,73697.0,0.011032,Black,4013,4013,Maricopa County AZ,...,52449.0,51638.25,21.5,"Maricopa County, AZ",21.5,49.89,672573,86788,0.344103,6.5234
12319,12312,"Maricopa County, AZ",4013.0,813.0,73697.0,0.011032,Black,4013,4013,Maricopa County AZ,...,9296.0,9312.8,8.6,"Maricopa County, AZ",8.6,23.46,182011,3484,0.076567,6.5234
12322,12312,"Maricopa County, AZ",4013.0,813.0,73697.0,0.011032,Black,4013,4013,Maricopa County AZ,...,52449.0,51638.25,21.5,"Maricopa County, AZ",21.5,49.89,672573,86788,0.344103,6.5234
12323,12312,"Maricopa County, AZ",4013.0,813.0,73697.0,0.011032,Black,4013,4013,Maricopa County AZ,...,9296.0,9312.8,8.6,"Maricopa County, AZ",8.6,23.46,182011,3484,0.076567,6.5234
12326,12313,"Maricopa County, AZ",4013.0,813.0,73697.0,0.011032,Black,4013,4013,Maricopa County AZ,...,52449.0,51638.25,21.5,"Maricopa County, AZ",21.5,49.89,672573,86788,0.344103,6.5234
12327,12313,"Maricopa County, AZ",4013.0,813.0,73697.0,0.011032,Black,4013,4013,Maricopa County AZ,...,9296.0,9312.8,8.6,"Maricopa County, AZ",8.6,23.46,182011,3484,0.076567,6.5234
12330,12313,"Maricopa County, AZ",4013.0,813.0,73697.0,0.011032,Black,4013,4013,Maricopa County AZ,...,52449.0,51638.25,21.5,"Maricopa County, AZ",21.5,49.89,672573,86788,0.344103,6.5234
12331,12313,"Maricopa County, AZ",4013.0,813.0,73697.0,0.011032,Black,4013,4013,Maricopa County AZ,...,9296.0,9312.8,8.6,"Maricopa County, AZ",8.6,23.46,182011,3484,0.076567,6.5234


12314      4013
12315      4013
12318      4013
12319      4013
12322      4013
12323      4013
12326      4013
12327      4013
12330      4013
12331      4013
12334      4013
12335      4013
12338      4013
12339      4013
12342      4013
12343      4013
12346      4013
12347      4013
12350      4013
12351      4013
12354      4013
12355      4013
12358      4013
12359      4013
12362      4013
12363      4013
12366      4013
12367      4013
12370      4013
12371      4013
          ...  
386202    55079
386203    55079
386204    55079
386205    55079
386206    55079
386207    55079
386208    55079
386209    55079
386210    55079
386211    55079
386212    55079
386213    55079
386214    55079
386215    55079
386216    55079
386217    55079
386218    55079
386219    55079
386220    55079
386221    55079
386222    55079
386223    55079
386224    55079
386225    55079
386226    55079
386227    55079
386228    55079
386229    55079
386230    55079
386231    55079
Name: fips_str, Length: 